In [28]:
from utils_az import FishEyeImage, rot
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u
from scipy.optimize import minimize, curve_fit
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, angular_separation, position_angle, offset_by, ICRS
from PIL import Image
from photutils.aperture import CircularAperture


In [29]:
file = 'img-z'

zhangbei = EarthLocation(lon=115*u.deg+14*u.arcsec,lat=41*u.deg+13*u.arcmin+53*u.arcsec,height = 1466)
dunhuang  = EarthLocation(lon=94.322799*u.deg,lat=40.359581*u.deg,height = 1100)
pic = FishEyeImage( file+'.CR3',loc = zhangbei)

solution = pic.solve(solve_size=1000)
_,_=pic.detect_stars_az()

{'RA': 141.37803150114797, 'Dec': 40.121001452814454, 'Roll': 168.86928012884306, 'FOV': 23.661560905884862, 'distortion': -0.006088866243518384, 'RMSE': 21.52299796131488, 'Matches': 26, 'Prob': 1.0193527449677233e-48, 'epoch_equinox': 2000, 'epoch_proper_motion': 2023.0, 'T_solve': 2.962299855425954, 'T_extract': 36.68619995005429}


In [30]:
pic.first_match(bin_size=5000, max_sep=40)


(array([ 299, 2712, 2669, ...,  299, 2669,    0], dtype=int64),
 <Angle [72.8346403 , 16.42306794, 18.01650294, ..., 70.35513429,
         20.85884596, 76.84805404] deg>)

In [31]:
img =  Image.open(file+'.jpg')
sep = angular_separation(pic.az*u.rad,pic.alt*u.rad,pic.catalog_skycoords.az,pic.catalog_skycoords.alt)
sep_constarit= sep<0.45*np.pi*u.rad
selected_cata_skyc = pic.catalog_skycoords[sep_constarit]
cata_x,cata_y,_,_ = pic.az_to_delta_xy(c_az=pic.az*u.rad, c_alt=pic.alt*u.rad, az=selected_cata_skyc.az, alt=selected_cata_skyc.alt, f=pic.f, k=pic.k, ks= pic.ks)

cata_xy = pic.delta_xy_to_xy(cata_x,cata_y,pic.c_x,pic.c_y, pic.az_roll)

%matplotlib qt

fig,ax = plt.subplots(1,1)
ax.imshow(img)
plt.tight_layout()

cata = CircularAperture(np.transpose(cata_xy), r=3)
cata.plot(color='red', lw=1)
plt.title('init')
plt.show()

In [32]:
pic.draw_residual(dpi=72, s=2, c = 'red',alpha=0.3)

In [33]:
init, result = pic.lens_optimize()
print(result)
x = result.x
pic.az = x[0]
pic.alt = x [1]
pic.az_roll = x[2]
pic.f = x[3]
pic.k = x[4]
pic.draw_residual(dpi=72, s=2, c = 'red',alpha=0.3)


  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 9.314083342403634
        x: [ 2.060e+00  1.527e+00  8.179e+00  1.459e+01 -1.662e-01]
      nit: 26
      jac: [ 9.362e-03  5.700e-02 -7.230e-03  1.357e-03 -1.162e-03]
     nfev: 198
     njev: 33
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>


In [34]:
pic.distort_optimize()

[-3.890181328802992e-08,
 1.3533620834252718e-06,
 -1.7286989500842283e-05,
 0.00012852565934182492,
 -0.0005683929810007211]

In [35]:
init, result = pic.plate_optimize()
print(init)
print(result)
x = result.x
pic.az = x[0]
pic.alt = x [1]
pic.az_roll = x[2]
pic.c_x=x[3]
pic.c_y = x[4]
pic.draw_residual(dpi=72, s=2, c = 'red',alpha=0.3)

[2.05961698e+00 1.52729022e+00 8.17871961e+00 3.00000000e+03
 2.00000000e+03]
  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 8.720468103683368
        x: [ 2.060e+00  1.527e+00  8.179e+00  3.000e+03  2.000e+03]
      nit: 11
      jac: [ 6.028e-03  4.442e-02  1.239e-02  6.328e-02 -2.367e-02]
     nfev: 90
     njev: 15
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>


In [36]:
img =  Image.open(file+'.jpg')
sep = angular_separation(pic.az*u.rad,pic.alt*u.rad,pic.catalog_skycoords.az,pic.catalog_skycoords.alt)
sep_constarit= sep<0.45*np.pi*u.rad
selected_cata_skyc = pic.catalog_skycoords[sep_constarit]
cata_x,cata_y,_,_ = pic.az_to_delta_xy(c_az=pic.az*u.rad, c_alt=pic.alt*u.rad, az=selected_cata_skyc.az, alt=selected_cata_skyc.alt, f=pic.f, k=pic.k, ks= pic.ks)

cata_xy = pic.delta_xy_to_xy(cata_x,cata_y,pic.c_x,pic.c_y, pic.az_roll)

%matplotlib qt
fig,ax = plt.subplots(1,1)
ax.imshow(img)
plt.tight_layout()
cata = CircularAperture(np.transpose(cata_xy), r=3)
cata.plot(color='red', lw=1)
plt.title('final')
plt.show()